In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import urllib.request
import tiktoken

from torch.utils.data import Dataset, DataLoader, Subset
from typing import List

In [2]:
# Gets the verdict
# url: str = "https://www.gutenberg.org/ebooks/67237.txt.utf-8"
# urllib.request.urlretrieve(url, "books/the-verdict.txt")

# Initializes the string that will contain the loaded text above
raw_text: str = None

# Reads the loaded text
with open("books/all.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()

# Logs the metadata of the text
print(f"Total number of characters: {len(raw_text)}")


Total number of characters: 18090147


In [3]:
# Initializes the tokenizer
tokenizer: tiktoken.core.Encoding = tiktoken.get_encoding("gpt2")

# Runs it on the raw text
tokens = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
print(f"Number of tokens: {len(tokens)}")
print(f"Vocabulary size: {tokenizer.n_vocab}")

Number of tokens: 4945337
Vocabulary size: 50257


In [4]:
# The below parameters define the shape of the embedding layer
vocabulary_size: int = tokenizer.n_vocab # => is the input size of the embedding layer, ie, the size of the vocabulary
token_embedding_dim: int = 1024 # => is the output size of the embedding layer

# The below parameters define the shape of the positional embedding layer
# The positional embedding layer is of shape context_size x output_dim
context_length: int = 16
output_dim: int = token_embedding_dim # => is the size of the output of the positional embedding,
                                      # that we want to be equal to the token_embedding_dim as this will be the input to the model

# The size of the sequence to be generated
max_new_tokens: int = 15

# The stride
stride: int = 1

# Batch size
batch_size: int = 256

# Learning rate
lr_rate: float = 0.000005

# Number of LSTM layers
num_layers: int = 6

# Number of epochs to train the model
num_epochs: int = 10

print(f"Vocabulary size set to: {vocabulary_size}")

Vocabulary size set to: 50257


In [5]:
# Example of using context with the tokenizer
print(f"The first 100 characters of the raw text:\n\n\"{raw_text[: 180]}\"\n")
print("Example of text generating with context_size slicing:")
for i in range(1, 4+1):
    context: int = tokens[: i]
    desired: int = tokens[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

The first 100 characters of the raw text:

"﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cos"

Example of text generating with context_size slicing:
� ----> �
� ----> �
﻿ ----> The
﻿The ---->  Project


In [6]:
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, context_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids) - context_length, stride):
            input_chunk = token_ids[i:i + context_length]
            self.input_ids.append(torch.tensor(input_chunk))

            # temp_id = token_ids[i + context_length]
            # target_tensor = torch.zeros([50257])
            # target_tensor[temp_id] = 1.0
            # self.target_ids.append(target_tensor)

            target_chunk = token_ids[i + 1: i + context_length + 1]            
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [7]:
def create_dataloader_v1(txt, batch_size=batch_size, context_length=context_length,
    stride=stride, shuffle=True, drop_last=True,
    num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, context_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers)
    
    return dataloader

In [8]:
device: str = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
class LSTMText(nn.Module):
    def __init__(
            self,
            id: int,
            input_size: int,
            hidden_size: int
    ):
        '''
        input_size: int -> size of the embedding dimension in the transformer parlance
        hidden_size: int -> size of the hidden state
        
        '''
        super().__init__()
        self.id: int = id
        self.input_size: int = input_size
        self.hidden_size: int = hidden_size

        # Input gate (I):
        # I_t = sigma(X_t.W_xi + H_t-1.W_hi + b_i)
        # Where:
        # X_t is X(the sequence) at time t
        # H_t-1 is H(the hidden state) at time t-1
        # W_xi is the weight matrix of X to I gate
        self.W_xi: nn.Parameter = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))

        # W_hi is the weight matrix of h to the I gate
        self.W_hi: torch.Tensor = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))

        # self.b_i is the bias to the I gate
        self.b_i: torch.Tensor = nn.Parameter(torch.Tensor( self.hidden_size))


        # Forget gate (F):
        # F_t = sigma(X_t.W_xf + H_t-1.W_hf + b_f)
        # Where:
        # X_t is X(the sequence) at time t
        # H_t-1 is H(the hidden state) at time t-1
        # W_xf is the weight matrix of X to F gate
        self.W_xf: nn.Parameter = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))

        # W_hf is the weight matrix of h to the F gate
        self.W_hf: torch.Tensor = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))

        # self.b_f is the bias to the F gate
        self.b_f: torch.Tensor = nn.Parameter(torch.Tensor( self.hidden_size))


        # Output gate (O):
        # O_t = sigma(X_t.W_xo + H_t-1.W_ho + b_o)
        # Where:
        # X_t is X(the sequence) at time t
        # H_t-1 is H(the hidden state) at time t-1
        # W_xo is the weight matrix of X to O gate
        self.W_xo: nn.Parameter = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))

        # W_ho is the weight matrix of h to the O gate
        self.W_ho: torch.Tensor = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))

        # self.b_o is the bias to the O gate
        self.b_o: torch.Tensor = nn.Parameter(torch.Tensor( self.hidden_size))

        
        # Cell (C):
        # C_t = sigma(X_t.W_xc + H_t-1.W_hc + b_c)
        # Where:
        # X_t is X(the sequence) at time t
        # H_t-1 is H(the hidden state) at time t-1
        # W_xc is the weight matrix of X to C cell
        self.W_xc: nn.Parameter = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))

        # W_ho is the weight matrix of h to the C cell
        self.W_hc: torch.Tensor = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))

        # self.b_o is the bias to the C cell
        self.b_c: torch.Tensor = nn.Parameter(torch.Tensor( self.hidden_size))

        # Initializes all weights 
        self.initialize_weights()

    def __repr__(self):
        repr: str = f"LSTMText(id={self.id}, input_size={self.input_size}, hidden_size={self.hidden_size})"
        return repr

    def initialize_weights(self):
        stdev: float = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdev, stdev)

    def forward(self, X: torch.Tensor, states: torch.Tensor = None):
        """
        assumes x.shape represents (batch_size, sequence_size, embedding_dimension)
        """
        bs, sequence_size, input_size = X.size()
        
        if input_size != self.input_size:
            raise ValueError(f"Input shape: {input_size} is not equal to model input size: {self.input_size}")

        if states is None:
            H_t, C_t = (
                torch.zeros(bs, self.hidden_size).to(device=X.device),
                torch.zeros(bs, self.hidden_size).to(device=X.device)
            )
        else:
            H_t, C_t = states

        outputs = []
        for t in range(sequence_size):
            x = X[:, t, :]
            # I is the input gate
            I_t = torch.sigmoid(torch.matmul(x, self.W_xi) + torch.matmul(H_t, self.W_hi) + self.b_i)

            # print(f"X shape: {X.shape}")
            # print(f"x shape: {x.shape}")
            # print(f"self.W_xi shape: {self.W_xi.shape}")
            # print(f"H_t shape: {H_t.shape}")
            # print(f"self.b_i shape: {self.b_i.shape}")
            # print(f"I_t shape: {I_t.shape}")

            # F is the forget state
            F_t = torch.sigmoid(torch.matmul(x, self.W_xf) + torch.matmul(H_t, self.W_hf) + self.b_f)

            # O is the output state
            O_t = torch.sigmoid(torch.matmul(x, self.W_xo) + torch.matmul(H_t, self.W_ho) + self.b_o)

            # C_t, the memory (C)ell is:
            # C_t = F(.)C_t-1 + I_t(.)C_temp
            # C_temp = tanh(X_t.W_xc + H_t-1.W_hc + b_c)
            C_temp = torch.tanh(torch.matmul(x, self.W_xc) + torch.matmul(H_t, self.W_hc) + self.b_c)
            C_t = F_t * C_t + I_t * C_temp
            H_t = O_t * torch.tanh(C_t)
            outputs.append(H_t.unsqueeze(1))

        result = torch.cat(outputs, dim=1)
        return result, (H_t, C_t)


In [10]:
class RNNModel(nn.Module):
    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 vocabulary_size: int,
                 context_length: int,
                 output_dim: int,
                 out_features: int,
                 num_layers: int
                 ):
        super().__init__()

        # The parameters
        self.vocabulary_size: int = vocabulary_size
        self.hidden_size: int = hidden_size
        self.input_size: int = input_size
        self.context_length: int = context_length
        self.output_dim: int = output_dim
        self.output_features: int = out_features
        self.num_layers: int = num_layers

        # the embedding layer
        self.token_embedding_layer: nn.Embedding = nn.Embedding(num_embeddings=self.vocabulary_size, embedding_dim=output_dim)

        # the positional embedding layer
        self.pos_embedding_layer: nn.Embedding = nn.Embedding(num_embeddings=self.context_length, embedding_dim=self.hidden_size)

        # the rnn (LSTM or GRU) rnn
        self.rnn_layers: nn.ModuleList = nn.ModuleList(
            [LSTMText(id=id, input_size=self.input_size, hidden_size=hidden_size) for id in range(self.num_layers)])

        # Linear layer to generate the output
        self.output_layer: nn.Linear = nn.Linear(in_features=input_size, out_features=out_features)

    def __repr__(self):
        repr: str = f'''RNNModel(input_size={self.input_size},
        hidden_size={self.hidden_size},
        vocabulary_size={self.vocabulary_size},
        context_length={self.context_length},
        output_dim={self.output_dim},
        out_features={self.output_features},
        (token_embedding_layer)={self.token_embedding_layer}),
        (pos_embedding_layer)={self.pos_embedding_layer},
        (rnn_layer)={self.rnn_layers},
        (output_layer)={self.output_layer}
        )
        '''
        return repr
    
    def forward(self, X: torch.Tensor):
        token_embeddings_ = self.token_embedding_layer(X)
        pos_embeddings_ = self.pos_embedding_layer(torch.arange(self.context_length).to(device=device))
        input_embeddings_ = token_embeddings_ + pos_embeddings_

        states = None
        for rnn_layer in self.rnn_layers:
            output_, states = rnn_layer(input_embeddings_, states)
        
        # print(f"Output from LSTM shape: {output_.shape}")
        output_ = self.output_layer(output_)
        # print(f"Output RNN shape: {output_.shape}")

        # output_ = torch.softmax(output_, dim=-1)
        # output_ = torch.argmax(output_, dim=-1)
        
        return output_


In [11]:
model = RNNModel(input_size=token_embedding_dim,
                 hidden_size=output_dim,
                 vocabulary_size=vocabulary_size,
                 context_length=context_length,
                 output_dim=output_dim,
                 out_features=vocabulary_size,
                 num_layers=num_layers)
model.to(device=device)
model.eval()

total_params: int = sum([p.numel() for p in model.parameters()])
print(f"Total number of parameters: {total_params}")

total_size_mb: int = (total_params * 4) / (1024 * 1024)
print("Total memory footprint required to"
          f" run the model: {total_size_mb:,.2f} MB")
model

Total number of parameters: 153349201
Total memory footprint required to run the model: 584.98 MB


RNNModel(input_size=1024,
        hidden_size=1024,
        vocabulary_size=50257,
        context_length=16,
        output_dim=1024,
        out_features=50257,
        (token_embedding_layer)=Embedding(50257, 1024)),
        (pos_embedding_layer)=Embedding(16, 1024),
        (rnn_layer)=ModuleList(
  (0): LSTMText(id=0, input_size=1024, hidden_size=1024)
  (1): LSTMText(id=1, input_size=1024, hidden_size=1024)
  (2): LSTMText(id=2, input_size=1024, hidden_size=1024)
  (3): LSTMText(id=3, input_size=1024, hidden_size=1024)
  (4): LSTMText(id=4, input_size=1024, hidden_size=1024)
  (5): LSTMText(id=5, input_size=1024, hidden_size=1024)
),
        (output_layer)=Linear(in_features=1024, out_features=50257, bias=True)
        )
        

In [12]:
def pad_or_trim_sequence(input: torch.Tensor, context_length: int):
    input = input[:, :context_length]
    output = F.pad(input, (0, context_length - input.shape[1]), "constant", 0)
    return output

In [13]:
def generate_text_simple(model,
                         idx: torch.Tensor,
                         max_new_tokens,
                         context_size):
    
    if idx.shape[-1] < context_size:
        idx = pad_or_trim_sequence(idx, context_length=context_size)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            output_ = model(idx_cond)
        
        output_ = torch.softmax(output_, dim=-1)
        output_ = torch.argmax(output_, dim=-1)
        # print(f"Output from model shape before cat: {output_.shape}")
        # print(f"output_[:, -1]: {output_[:, 2].unsqueeze(1).shape}")
        # print(f"Input generate_text_simple idx: {idx.shape}")
        idx = torch.cat((idx, output_[:, -1].unsqueeze(1)), dim=-1)
    return idx

In [14]:
def text_to_token_ids(texts, tokenizer):
    encoded_tensors = []
    for text in texts:
        encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})    
        encoded_tensor = torch.tensor(encoded).unsqueeze(0)
        encoded_tensors.append(encoded_tensor)
    return_tensor = torch.cat(encoded_tensors, dim=0).to(device=device)
    return return_tensor

def token_ids_to_text(token_ids, tokenizer):
    response = []
    for batch in range(token_ids.shape[0]):
        response.append(
            tokenizer.decode(
                token_ids[batch, :].squeeze(0).tolist()
                )
            )
    return response

In [15]:
start_context = ["He was always thinking"] #He was always thinking what the estate would be if those mortgages could but be paid off.
tokenizer = tiktoken.get_encoding("gpt2")
# every effort moves you

In [16]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=max_new_tokens,
    context_size=context_length
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 ['He was always thinking!!!!!!!!!!!!urrectionPatパsterdamBritish35 DemocratPatパ35 outrage�Catalog!? ninety']


In [ ]:

# 1.2 Generator to enabling split dataset into train and validation subsets
generator1: torch.Generator = torch.Generator().manual_seed(918)

dataset = GPTDatasetV1(raw_text, tokenizer, context_length, stride)

# 1.3 Creates a list with both subsets, 90% training, 10% evaluation
datasets: List[Subset] = torch.utils.data.random_split(
    dataset,
    [0.9, 0.1],
    generator=generator1
)

# 1.4 Assigns train and validation datasets accordingly
train_dataset: Subset = datasets[0]
validation_dataset: Subset = datasets[1]

train_dataloader: DataLoader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=2
    )

val_dataloader = DataLoader(
        validation_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=2
    )

num_batches: int = len(train_dataloader)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr_rate)
evaluator = nn.CrossEntropyLoss()

In [ ]:
counter = 0
model.train()
for epoch in range(num_epochs):

    for input_batch, target_batch in train_dataloader:
        optimizer.zero_grad()

        input_batch = input_batch.to(device=device)
        target_batch = target_batch.to(device=device)
        output = model(input_batch)

        output_flatten: torch.Tensor = torch.flatten(output, 0, 1)
        target_batch_flatten: torch.Tensor = torch.flatten(target_batch, 0, 1)
        # print(f"Model output flatten shape: {output_flatten.shape}")
        # print(f"target_batch flatten shape: {target_batch_flatten.shape}")
        loss: torch.Tensor = evaluator(output_flatten, target_batch_flatten)

        loss.backward()
        optimizer.step()

        if counter % 10000 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.6f} Batches: {counter}/{num_batches}')
        counter += 1

    counter = 0

    model.eval()
    token_ids = generate_text_simple(
                    model=model,
                    idx=text_to_token_ids(start_context, tokenizer),
                    max_new_tokens=max_new_tokens,
                    context_size=context_length
                )
    print("Output text: ", token_ids_to_text(token_ids, tokenizer))
    model.train()
    print(f"End of epoch: {epoch} Loss: {loss:.6f}")